<a href="https://colab.research.google.com/github/be-mich/LIGN-167-Project/blob/main/Experiment_4_(seasons).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook continues with the third experiment. Here I experiment with a custom Season Sampler. This Data loader will load data by season, and within each season shuffle the training the data. That is, season 1 of the training data will be loaded first and so on. This data loader is less random that previous data loaders. Here I am experimenting if the the scripts have a progression to the development of George Costanza, then maybe this progression should be preserved for the model to understand. 

In [ ]:
#setup 
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks")

In [ ]:
# all the imports

import glob
import logging
import os
import pickle
import random
import re
import shutil
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd


from sklearn.model_selection import train_test_split

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler, Sampler #for our custom
from torch.utils.data.distributed import DistributedSampler
from tqdm.notebook import tqdm, trange

from pathlib import Path

!pip install transformers

from transformers import (
    MODEL_WITH_LM_HEAD_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    #SGD, #try other optimizer- not available in transformers
    AutoConfig,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)


try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter
# Configs
logger = logging.getLogger(__name__)

MODEL_CONFIG_CLASSES = list(MODEL_WITH_LM_HEAD_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

     |████████████████████████████████| 3.8 MB 8.2 MB/s 
     |████████████████████████████████| 895 kB 51.0 MB/s 
     |████████████████████████████████| 596 kB 57.9 MB/s 
     |████████████████████████████████| 6.5 MB 50.0 MB/s 
     |████████████████████████████████| 67 kB 6.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# Args to allow for easy convertion of python script to notebook
class Args():
    def __init__(self):
        self.output_dir = 'output-small'
        self.model_type = 'gpt2'
        self.model_name_or_path = 'microsoft/DialoGPT-small'
        self.config_name = 'microsoft/DialoGPT-small'
        self.tokenizer_name = 'microsoft/DialoGPT-small'
        self.cache_dir = 'cached'
        self.block_size = 512
        self.do_train = True
        self.do_eval = True
        self.evaluate_during_training = False
        self.per_gpu_train_batch_size = 4
        self.per_gpu_eval_batch_size = 4
        self.gradient_accumulation_steps = 1
        self.learning_rate = 5e-5
        self.weight_decay = 0.0
        self.adam_epsilon = 1e-8
        self.max_grad_norm = 1.0
        self.num_train_epochs = 3
        self.max_steps = -1
        self.warmup_steps = 0
        self.logging_steps = 1000
        self.save_steps = 3500
        self.save_total_limit = None
        self.eval_all_checkpoints = False
        self.no_cuda = False
        self.overwrite_output_dir = True
        self.overwrite_cache = True
        self.should_continue = False
        self.seed = 42
        self.local_rank = -1
        self.fp16 = False
        self.fp16_opt_level = 'O1'

args = Args()

Import Seinfeld scripts data from google drive (previously obtained from Kaggle)

In [ ]:
! gdown https://drive.google.com/uc?id=1wSKl_642G6lsk4AmrYXpa_8XWb4hcY_B

In [ ]:
seinfeld_scripts = pd.read_csv("./scripts.csv")

In [ ]:
seinfeld_scripts.head()

,Unnamed: 0,Character,Dialogue,EpisodeNo,SEID,Season
0,0,JERRY,Do you know what this is all about? Do you kno...,1.0,S01E01,1.0
1,1,JERRY,"(pointing at Georges shirt) See, to me, that b...",1.0,S01E01,1.0
2,2,GEORGE,Are you through?,1.0,S01E01,1.0
3,3,JERRY,"You do of course try on, when you buy?",1.0,S01E01,1.0
4,4,GEORGE,"Yes, it was purple, I liked it, I dont actuall...",1.0,S01E01,1.0


In [ ]:
#make df
type(seinfeld_scripts)

pandas.core.frame.DataFrame

In [ ]:
#check for empty dialogue
seinfeld_scripts[seinfeld_scripts['Dialogue'].isnull()==True]


,Unnamed: 0,Character,Dialogue,EpisodeNo,SEID,Season
13529,13529,** Pies - Just in case you did not know what t...,NaN,7.0,S04E07,4.0
14938,14938,"[On the bright side, Kramer and George arrive ...",NaN,12.0,S04E12,4.0
18353,18353,(The show begins. There are three different se...,NaN,24.0,S04E24,4.0
18354,18354,(Jerry's doing his stand-up routine at a comed...,NaN,24.0,S04E24,4.0
18355,18355,"We see the title 'Jerry', then, sitting at the...",NaN,24.0,S04E24,4.0
42939,42939,Definitions of several items in the Chicken Ro...,NaN,8.0,S08E08,8.0
45847,45847,HAL,NaN,18.0,S08E18,8.0
49651,49651,Notice,NaN,8.0,S09E08,9.0
50013,50013,The definition of Sari or Saree is,NaN,8.0,S09E08,9.0
53142,53142,MONTAGE,NaN,18.0,S09E18,9.0


In [ ]:
#drop rows with missing values

seinfeld_scripts = seinfeld_scripts.dropna().reset_index(drop=True)
print(len(seinfeld_scripts))

54606


In [ ]:
seinfeld_scripts['Season'].value_counts()

7.0    7806
9.0    7293
4.0    7285
5.0    7228
6.0    6903
8.0    6895
3.0    6811
2.0    3077
1.0    1308
Name: Season, dtype: int64

In [ ]:
#subset data by season  

seinfeld_season1 = seinfeld_scripts[seinfeld_scripts['Season']==1.0]

seinfeld_season2 = seinfeld_scripts[seinfeld_scripts['Season']==2.0]

seinfeld_season3 = seinfeld_scripts[seinfeld_scripts['Season']==3.0]

seinfeld_season4 = seinfeld_scripts[seinfeld_scripts['Season']==4.0]

seinfeld_season5 = seinfeld_scripts[seinfeld_scripts['Season']==5.0]

seinfeld_season6 = seinfeld_scripts[seinfeld_scripts['Season']==6.0]

seinfeld_season7 = seinfeld_scripts[seinfeld_scripts['Season']==7.0]

seinfeld_season8 = seinfeld_scripts[seinfeld_scripts['Season']==8.0]

seinfeld_season9 = seinfeld_scripts[seinfeld_scripts['Season']==9.0]

Clean up data 

In [ ]:
seinfeld_season1 = seinfeld_season1.drop(columns=['Unnamed: 0','EpisodeNo','SEID','Season'])
seinfeld_season2 = seinfeld_season2.drop(columns=['Unnamed: 0','EpisodeNo','SEID','Season'])
seinfeld_season3 = seinfeld_season3.drop(columns=['Unnamed: 0','EpisodeNo','SEID','Season'])
seinfeld_season4 = seinfeld_season4.drop(columns=['Unnamed: 0','EpisodeNo','SEID','Season'])
seinfeld_season5 = seinfeld_season5.drop(columns=['Unnamed: 0','EpisodeNo','SEID','Season'])
seinfeld_season6 = seinfeld_season6.drop(columns=['Unnamed: 0','EpisodeNo','SEID','Season'])
seinfeld_season7 = seinfeld_season7.drop(columns=['Unnamed: 0','EpisodeNo','SEID','Season'])
seinfeld_season8 = seinfeld_season8.drop(columns=['Unnamed: 0','EpisodeNo','SEID','Season'])
seinfeld_season9 = seinfeld_season9.drop(columns=['Unnamed: 0','EpisodeNo','SEID','Season'])

In [ ]:
#can get rid of some columns- we only need character and dialogue
seinfeld_scripts = seinfeld_scripts.drop(columns=['Unnamed: 0','EpisodeNo','SEID','Season'])

In [ ]:
seinfeld_scripts.head()

,Character,Dialogue
0,JERRY,Do you know what this is all about? Do you kno...
1,JERRY,"(pointing at Georges shirt) See, to me, that b..."
2,GEORGE,Are you through?
3,JERRY,"You do of course try on, when you buy?"
4,GEORGE,"Yes, it was purple, I liked it, I dont actuall..."


In [ ]:
len(seinfeld_scripts)

54606

In [ ]:
sum(seinfeld_scripts.Character == 'GEORGE')

9708

In [ ]:
NAME = 'GEORGE'

First restructure dataframe so that we have George's dialogue as a response, and we have 10 previous exchanges of dialogue as context. 

In [ ]:
contexted = []
#context window of size 4
#or window of 10
#n = 4
n = 10

for i in seinfeld_scripts[seinfeld_scripts.Character == NAME].index:
    if i < n:
        continue
    row = []
    prev = i - 1 - n #subtract 1 so row contains current and 4/ or 10 previous responses
    for j in range(i, prev, -1):
        row.append(seinfeld_scripts.Dialogue[j])
    contexted.append(row)
columns = ['response','context']
columns = columns + ['context/' + str(i) for i in range(n-1)]

formatted_df = pd.DataFrame.from_records(contexted, columns=columns)
    

In [ ]:
#function to run above on each dataframe 

def contextualize_data(dataset):
  contexted =[]
  n=10

  for i in dataset[dataset.Character == NAME].index:
    if i < n:
      continue
    row = []
    prev = i - 1 - n #
    for j in range(i, prev, -1):
      row.append(dataset.Dialogue[j])
    contexted.append(row)
  columns = ['response','context']
  columns = columns + ['context/' + str(i) for i in range(n-1)]

  return pd.DataFrame.from_records(contexted, columns=columns)

In [ ]:
formatted_season1 = contextualize_data(seinfeld_season1)
#formatted_season1
#index 0:256
formatted_season2 = contextualize_data(seinfeld_season2.reset_index())
formatted_season2
#index 0:637
formatted_season3 = contextualize_data(seinfeld_season3.reset_index())
formatted_season3
#index 0:1462
formatted_season4 = contextualize_data(seinfeld_season4.reset_index())
formatted_season4
#index 0:1439
formatted_season5 = contextualize_data(seinfeld_season5.reset_index())
formatted_season5
#index 0:1202
formatted_season6 = contextualize_data(seinfeld_season6.reset_index())
formatted_season6
#index 0:1081
formatted_season7 = contextualize_data(seinfeld_season7.reset_index())
formatted_season7
#index 0:1318
formatted_season8 = contextualize_data(seinfeld_season8.reset_index())
formatted_season8
#index 0:1024
formatted_season9 = contextualize_data(seinfeld_season9.reset_index())
formatted_season9
#index 0:1242

,response,context,context/0,context/1,context/2,context/3,context/4,context/5,context/6,context/7,context/8
0,What if we grew muttonchops?,Because you said this would be better. Remembe...,Well why didn't we?,"I told you, we should have taken some kind of ...","I know, I hate it too. I feel like an out of w...",(picking at his moustache) You know I cannot s...,Then who are the Dutch?,Holland *is* the Netherlands.,"No, that's the Netherlands.","(also wearing a moustache) What do you mean, '...",What is Holland?
1,Buzz cuts? Parachute pants!,No.,What if we grew muttonchops?,Because you said this would be better. Remembe...,Well why didn't we?,"I told you, we should have taken some kind of ...","I know, I hate it too. I feel like an out of w...",(picking at his moustache) You know I cannot s...,Then who are the Dutch?,Holland *is* the Netherlands.,"No, that's the Netherlands."
2,(resigned) Dammit.,"Stop it, George. Stop it. I'm sorry, you've go...",Buzz cuts? Parachute pants!,No.,What if we grew muttonchops?,Because you said this would be better. Remembe...,Well why didn't we?,"I told you, we should have taken some kind of ...","I know, I hate it too. I feel like an out of w...",(picking at his moustache) You know I cannot s...,Then who are the Dutch?
3,"Hey hey hey, check me out, huh?",(resigned) Dammit.,"Stop it, George. Stop it. I'm sorry, you've go...",Buzz cuts? Parachute pants!,No.,What if we grew muttonchops?,Because you said this would be better. Remembe...,Well why didn't we?,"I told you, we should have taken some kind of ...","I know, I hate it too. I feel like an out of w...",(picking at his moustache) You know I cannot s...
4,"Yeah, with crutches everyone has questions.","No more crutches, that must be a relief.","Hey hey hey, check me out, huh?",(resigned) Dammit.,"Stop it, George. Stop it. I'm sorry, you've go...",Buzz cuts? Parachute pants!,No.,What if we grew muttonchops?,Because you said this would be better. Remembe...,Well why didn't we?,"I told you, we should have taken some kind of ..."
...,...,...,...,...,...,...,...,...,...,...,...
1238,What?,Three dollars!,"Yes, please.",You want bread?,"Um, excuse me, I think you forgot my bread.",Just forget it. Let it go.,I didn't get any bread.,Medium crab bisque.,Medium turkey chili.,They used to come to my restaurant.,How do you know the defendants?
1239,You had to hop! You had to hop on the plane.,"Order! Order in this court, I will clear this ...",We find the defendants - guilty.,Will the defendants please rise. And how do yo...,"We have, Your Honor.","Ladies and gentlemen of the jury, have you rea...",All rise.,Maybe because she knows you're going to jail.,I think it's going to be okay - that girl just...,I've always loved ... United Airlines.,"Hey Elaine, what was it you were about to say ..."
1240,Really?,"See now, to me, that button is in the worst po...","I got it - it's out! How about that, huh? Oh, ...",I think that would be a very nice gesture.,Cause you only get one call. The prison call i...,Sure.,"If I call Jill from prison, do you think that ...",Why don't you just blow it out your a...,Could be fun. Don't have to worry about your m...,"Well, it's only a year. That's not so bad. We'...","Oh, and by the way, they're real, and they're ..."
1241,Haven't we had this conversation before?,Oh yeah. The second button is the key button. ...,Really?,"See now, to me, that button is in the worst po...","I got it - it's out! How about that, huh? Oh, ...",I think that would be a very nice gesture.,Cause you only get one call. The prison call i...,Sure.,"If I call Jill from prison, do you think that ...",Why don't you just blow it out your a...,Could be fun. Don't have to worry about your m...


In [ ]:
#train test split each season individually. 
from sklearn.model_selection import train_test_split
trn_df_1, test_df_1 = train_test_split(formatted_season1, test_size = 0.2, shuffle=False)
trn_df_2, test_df_2 = train_test_split(formatted_season2, test_size = 0.2, shuffle=False)
trn_df_3, test_df_3 = train_test_split(formatted_season3, test_size = 0.2, shuffle=False)
trn_df_4, test_df_4 = train_test_split(formatted_season4, test_size = 0.2, shuffle=False)
trn_df_5, test_df_5 = train_test_split(formatted_season5, test_size = 0.2, shuffle=False)
trn_df_6, test_df_6 = train_test_split(formatted_season6, test_size = 0.2, shuffle=False)
trn_df_7, test_df_7 = train_test_split(formatted_season7, test_size = 0.2, shuffle=False)
trn_df_8, test_df_8 = train_test_split(formatted_season8, test_size = 0.2, shuffle=False)
trn_df_9, test_df_9 = train_test_split(formatted_season9, test_size = 0.2, shuffle=False)

In [ ]:
#grab indices of training datasets
season1_index = len(trn_df_1) #205
season2_index = len(trn_df_2)
season3_index = len(trn_df_3)
season4_index = len(trn_df_4)
season5_index = len(trn_df_5)
season6_index = len(trn_df_6)
season7_index = len(trn_df_7)
season8_index = len(trn_df_8)
season9_index = len(trn_df_9)

print(season1_index, season2_index)
trn_df_2

205 509


,response,context,context/0,context/1,context/2,context/3,context/4,context/5,context/6,context/7,context/8
0,And I didnt even ask her out. She asked me out...,Youre a human being.,"Oh, I had no choice. She squeezed it out of me...",Did you tell her you loved her?,"I kept looking at her face. Id go, Cmon, love ...",You tried.,It just didnt work out. What can I do? I wante...,Everyday.,People break up all the time.,"No, of course not.",She cant kill me right?
1,I am. Im a nice guy. And she seduced me! We we...,Youre too nice a guy.,And I didnt even ask her out. She asked me out...,Youre a human being.,"Oh, I had no choice. She squeezed it out of me...",Did you tell her you loved her?,"I kept looking at her face. Id go, Cmon, love ...",You tried.,It just didnt work out. What can I do? I wante...,Everyday.,People break up all the time.
2,I had nothing to do wtih any of this! I met al...,Youre flesh and blood.,I am. Im a nice guy. And she seduced me! We we...,Youre too nice a guy.,And I didnt even ask her out. She asked me out...,Youre a human being.,"Oh, I had no choice. She squeezed it out of me...",Did you tell her you loved her?,"I kept looking at her face. Id go, Cmon, love ...",You tried.,It just didnt work out. What can I do? I wante...
3,My back is killing me.,I told you when I met her.,I had nothing to do wtih any of this! I met al...,Youre flesh and blood.,I am. Im a nice guy. And she seduced me! We we...,Youre too nice a guy.,And I didnt even ask her out. She asked me out...,Youre a human being.,"Oh, I had no choice. She squeezed it out of me...",Did you tell her you loved her?,"I kept looking at her face. Id go, Cmon, love ..."
4,"Oh yeah, everybodys guy is the best.","You gotta go to my chiropractor, hes the best.",My back is killing me.,I told you when I met her.,I had nothing to do wtih any of this! I met al...,Youre flesh and blood.,I am. Im a nice guy. And she seduced me! We we...,Youre too nice a guy.,And I didnt even ask her out. She asked me out...,Youre a human being.,"Oh, I had no choice. She squeezed it out of me..."
...,...,...,...,...,...,...,...,...,...,...,...
504,God! She's cooking dinner for God! She's yelli...,"Oh, and she cooks dinner onstage for some cele...",She dragged me down to that warehouse on the w...,(Gesturing to George) You believe this guy? He...,"(Again, sarcastic) Oh, what a huge blow to the...",She stopped performing.,(Sarcastic) Maybe for her new performance piec...,"Thin ice, George. Thin ice..","What, you don't think I can attract beautiful,...","Oh, pardon me for trying to set you up with a ...",That woman was unequivocally the worst date of...
505,"Oh, yeah, sure, accident, right. She was aimin...",It was an accident!,God! She's cooking dinner for God! She's yelli...,"Oh, and she cooks dinner onstage for some cele...",She dragged me down to that warehouse on the w...,(Gesturing to George) You believe this guy? He...,"(Again, sarcastic) Oh, what a huge blow to the...",She stopped performing.,(Sarcastic) Maybe for her new performance piec...,"Thin ice, George. Thin ice..","What, you don't think I can attract beautiful,..."
506,"And then, then, then she leaves with somebody ...",(To Elaine) he thought he still had a shot.,"Oh, yeah, sure, accident, right. She was aimin...",It was an accident!,God! She's cooking dinner for God! She's yelli...,"Oh, and she cooks dinner onstage for some cele...",She dragged me down to that warehouse on the w...,(Gesturing to George) You believe this guy? He...,"(Again, sarcastic) Oh, what a huge blow to the...",She stopped performing.,(Sarcastic) Maybe for her new performance piec...
507,I still have it. The collar's okay. I wear it ...,What ever happened with the shirt?,"And then, then, then she leaves with somebody ...",(To Elaine) he thought he still had a shot.,"Oh, yeah, sure, accident, right. She was aimin...",It was an accident!,God! She's cooking dinner for God! She's yelli...,"Oh, and she cooks dinner onstage for some cele...",She dragged me do

In [ ]:
#concat these

trn_df_all_seasons = pd.concat([trn_df_1, trn_df_2, trn_df_3, trn_df_4,
                               trn_df_5, trn_df_6, trn_df_7,
                               trn_df_8, trn_df_9],ignore_index = True)

test_all_seasons = pd.concat([test_df_1, test_df_2, test_df_3, test_df_4,
                             test_df_5, test_df_6, test_df_7,
                             test_df_8, test_df_9], ignore_index = True)



In [ ]:
trn_df_all_seasons.iloc[205:714]

,response,context,context/0,context/1,context/2,context/3,context/4,context/5,context/6,context/7,context/8
205,And I didnt even ask her out. She asked me out...,Youre a human being.,"Oh, I had no choice. She squeezed it out of me...",Did you tell her you loved her?,"I kept looking at her face. Id go, Cmon, love ...",You tried.,It just didnt work out. What can I do? I wante...,Everyday.,People break up all the time.,"No, of course not.",She cant kill me right?
206,I am. Im a nice guy. And she seduced me! We we...,Youre too nice a guy.,And I didnt even ask her out. She asked me out...,Youre a human being.,"Oh, I had no choice. She squeezed it out of me...",Did you tell her you loved her?,"I kept looking at her face. Id go, Cmon, love ...",You tried.,It just didnt work out. What can I do? I wante...,Everyday.,People break up all the time.
207,I had nothing to do wtih any of this! I met al...,Youre flesh and blood.,I am. Im a nice guy. And she seduced me! We we...,Youre too nice a guy.,And I didnt even ask her out. She asked me out...,Youre a human being.,"Oh, I had no choice. She squeezed it out of me...",Did you tell her you loved her?,"I kept looking at her face. Id go, Cmon, love ...",You tried.,It just didnt work out. What can I do? I wante...
208,My back is killing me.,I told you when I met her.,I had nothing to do wtih any of this! I met al...,Youre flesh and blood.,I am. Im a nice guy. And she seduced me! We we...,Youre too nice a guy.,And I didnt even ask her out. She asked me out...,Youre a human being.,"Oh, I had no choice. She squeezed it out of me...",Did you tell her you loved her?,"I kept looking at her face. Id go, Cmon, love ..."
209,"Oh yeah, everybodys guy is the best.","You gotta go to my chiropractor, hes the best.",My back is killing me.,I told you when I met her.,I had nothing to do wtih any of this! I met al...,Youre flesh and blood.,I am. Im a nice guy. And she seduced me! We we...,Youre too nice a guy.,And I didnt even ask her out. She asked me out...,Youre a human being.,"Oh, I had no choice. She squeezed it out of me..."
...,...,...,...,...,...,...,...,...,...,...,...
709,God! She's cooking dinner for God! She's yelli...,"Oh, and she cooks dinner onstage for some cele...",She dragged me down to that warehouse on the w...,(Gesturing to George) You believe this guy? He...,"(Again, sarcastic) Oh, what a huge blow to the...",She stopped performing.,(Sarcastic) Maybe for her new performance piec...,"Thin ice, George. Thin ice..","What, you don't think I can attract beautiful,...","Oh, pardon me for trying to set you up with a ...",That woman was unequivocally the worst date of...
710,"Oh, yeah, sure, accident, right. She was aimin...",It was an accident!,God! She's cooking dinner for God! She's yelli...,"Oh, and she cooks dinner onstage for some cele...",She dragged me down to that warehouse on the w...,(Gesturing to George) You believe this guy? He...,"(Again, sarcastic) Oh, what a huge blow to the...",She stopped performing.,(Sarcastic) Maybe for her new performance piec...,"Thin ice, George. Thin ice..","What, you don't think I can attract beautiful,..."
711,"And then, then, then she leaves with somebody ...",(To Elaine) he thought he still had a shot.,"Oh, yeah, sure, accident, right. She was aimin...",It was an accident!,God! She's cooking dinner for God! She's yelli...,"Oh, and she cooks dinner onstage for some cele...",She dragged me down to that warehouse on the w...,(Gesturing to George) You believe this guy? He...,"(Again, sarcastic) Oh, what a huge blow to the...",She stopped performing.,(Sarcastic) Maybe for her new performance piec...
712,I still have it. The collar's okay. I wear it ...,What ever happened with the shirt?,"And then, then, then she leaves with somebody ...",(To Elaine) he thought he still had a shot.,"Oh, yeah, sure, accident, right. She was aimin...",It was an accident!,God! She's cooking dinner for God! She's yelli...,"Oh, and she cooks dinner onstage for some cele...",She dra

In [ ]:
print(len(formatted_df))

9704


In [ ]:
#see if i have  na

formatted_df.isnull().values.any()

False

In [ ]:
#preprocess tokenizer
!pip install transformers
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/641 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Here I will experiment with a custom batch sampler. This specific custom batch sampler will sample and batch indices belonging to respective seasons. This will test the hypothesis that each season the development of the characters is different. That is, George from each season is slightly different and in order to understand his character we should have the model fed this development rather than all at once.

In [ ]:
print(season1_index)
print(season2_index)
print(season1_index + season2_index)

205
509
714


In [ ]:
#custom sampler for seasons

class SeasonSampler(Sampler):
  def __init__(self, all_seasons_formatted):
    self.season1_indices = list(range(0, season1_index))
    self.season2_indices = list(range(season1_index+1, season1_index + season2_index +1 ))
    self.season3_indices = list(range(season2_index+1, season2_index + season3_index +1))
    self.season4_indices = list(range(season3_index+1, season3_index + season4_index +1))
    self.season5_indices = list(range(season4_index+1, season4_index + season5_index +1))
    self.season6_indices = list(range(season5_index+1, season5_index + season6_index +1))
    self.season7_indices = list(range(season6_index + 1, season6_index + season7_index +1))
    self.season8_indices = list(range(season7_index + 1, season7_index + season8_index +1))
    self.season9_indices = list(range(season8_index + 1, season8_index + season9_index +1))
    #self.batch_size = batch_size
  def __iter__(self):
    random.shuffle(self.season1_indices)
    random.shuffle(self.season2_indices)
    random.shuffle(self.season3_indices)
    random.shuffle(self.season4_indices)
    random.shuffle(self.season5_indices)
    random.shuffle(self.season6_indices)
    random.shuffle(self.season7_indices)
    random.shuffle(self.season8_indices)
    random.shuffle(self.season9_indices)

    return iter(self.season1_indices + self.season2_indices +
                self.season3_indices + self.season4_indices +
                self.season5_indices + self.season6_indices +
                self.season6_indices + self.season7_indices +
                self.season8_indices + self.season9_indices)


  def __len__(self):
    return (len(self.season1_indices) + len(self.season2_indices)+
            len(self.season3_indices)+ len(self.season4_indices)+
            len(self.season5_indices)+ len(self.season6_indices)+
            len(self.season7_indices)+len(self.season8_indices) + len(self.season9_indices)) 
            


In [ ]:
import torch

In [ ]:
# create dataset suitable for our model
def construct_conv(row, tokenizer, eos = True):
    flatten = lambda l: [item for sublist in l for item in sublist]
    conv = list(reversed([tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row]))
    conv = flatten(conv)
    return conv

class ConversationDataset(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, args, df, block_size=512):

        block_size = block_size - (tokenizer.model_max_length - tokenizer.max_len_single_sentence)

        directory = args.cache_dir
        cached_features_file = os.path.join(
            directory, args.model_type + "_cached_lm_" + str(block_size)
        )

        if os.path.exists(cached_features_file) and not args.overwrite_cache:
            logger.info("Loading features from cached file %s", cached_features_file)
            with open(cached_features_file, "rb") as handle:
                self.examples = pickle.load(handle)
        else:
            logger.info("Creating features from dataset file at %s", directory)

            self.examples = []
            for _, row in df.iterrows():
                conv = construct_conv(row, tokenizer)
                self.examples.append(conv)

            logger.info("Saving features into cached file %s", cached_features_file)
            with open(cached_features_file, "wb") as handle:
                pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item], dtype=torch.long)

In [ ]:
# Cacheing and storing of data/checkpoints

def load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False):
    return ConversationDataset(tokenizer, args, df_val if evaluate else df_trn)


def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


def _sorted_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> List[str]:
    ordering_and_checkpoint_path = []

    glob_checkpoints = glob.glob(os.path.join(args.output_dir, "{}-*".format(checkpoint_prefix)))

    for path in glob_checkpoints:
        if use_mtime:
            ordering_and_checkpoint_path.append((os.path.getmtime(path), path))
        else:
            regex_match = re.match(".*{}-([0-9]+)".format(checkpoint_prefix), path)
            if regex_match and regex_match.groups():
                ordering_and_checkpoint_path.append((int(regex_match.groups()[0]), path))

    checkpoints_sorted = sorted(ordering_and_checkpoint_path)
    checkpoints_sorted = [checkpoint[1] for checkpoint in checkpoints_sorted]
    return checkpoints_sorted


def _rotate_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> None:
    if not args.save_total_limit:
        return
    if args.save_total_limit <= 0:
        return

    # Check if we should delete older checkpoint(s)
    checkpoints_sorted = _sorted_checkpoints(args, checkpoint_prefix, use_mtime)
    if len(checkpoints_sorted) <= args.save_total_limit:
        return

    number_of_checkpoints_to_delete = max(0, len(checkpoints_sorted) - args.save_total_limit)
    checkpoints_to_be_deleted = checkpoints_sorted[:number_of_checkpoints_to_delete]
    for checkpoint in checkpoints_to_be_deleted:
        logger.info("Deleting older checkpoint [{}] due to args.save_total_limit".format(checkpoint))
        shutil.rmtree(checkpoint)

In [ ]:
from transformers import AutoModelWithLMHead, AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
model = AutoModelWithLMHead.from_pretrained("microsoft/DialoGPT-small")

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:882: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/335M [00:00<?, ?B/s]

In [ ]:
def train(args, train_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer) -> Tuple[int, float]:
    """ Train the model """
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)
#here are the different possible data loaders
#for random sampler
    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
#for sequential sampler
    #seq_sampler = SequentialSampler(train_dataset)
#for season sampler
    season_sampler = SeasonSampler(train_dataset)
#dataloader
    train_dataloader = DataLoader(
        train_dataset, 
        sampler = season_sampler,
        #sampler = seq_sampler,
        #sampler = half_sampler,
        #sampler=train_sampler, 
        batch_size=args.train_batch_size, 
        collate_fn=collate , drop_last = True
    )

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    model = model.module if hasattr(model, "module") else model  # Take care of distributed/parallel training
    model.resize_token_embeddings(len(tokenizer))
    # add_special_tokens_(model, tokenizer)


    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    #optimizer = SGD(optimizer_grouped_parameters, lr=args.learning_rate, momentum=0.9)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    if (
        args.model_name_or_path
        and os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt"))
        and os.path.isfile(os.path.join(args.model_name_or_path, "scheduler.pt"))
    ):
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # multi-gpu training (should be after apex fp16 initialization)
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Distributed training (should be after apex fp16 initialization)
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model, device_ids=[args.local_rank], output_device=args.local_rank, find_unused_parameters=True
        )

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if args.model_name_or_path and os.path.exists(args.model_name_or_path):
        try:
            # set global_step to gobal_step of last saved checkpoint from model path
            checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
            steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

            logger.info("  Continuing training from checkpoint, will skip to saved global_step")
            logger.info("  Continuing training from epoch %d", epochs_trained)
            logger.info("  Continuing training from global step %d", global_step)
            logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
        except ValueError:
            logger.info("  Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0

    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0]
    )
    set_seed(args)  # Added here for reproducibility
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):

            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            inputs, labels = (batch, batch)
            if inputs.shape[1] > 1024: continue
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            model.train()
            outputs = model(inputs, labels=labels)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            if args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics
                    if (
                        args.local_rank == -1 and args.evaluate_during_training
                    ):  # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(args, model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar("eval_{}".format(key), value, global_step)
                    tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar("loss", (tr_loss - logging_loss) / args.logging_steps, global_step)
                    logging_loss = tr_loss

                if args.local_rank in [-1, 0] and args.save_steps > 0 and global_step % args.save_steps == 0:
                    checkpoint_prefix = "checkpoint"
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, "{}-{}".format(checkpoint_prefix, global_step))
                    os.makedirs(output_dir, exist_ok=True)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)

                    _rotate_checkpoints(args, checkpoint_prefix)

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    logger.info("Saving optimizer and scheduler states to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    if args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / global_step

# Evaluation of some model

def evaluate(args, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, df_trn, df_val, prefix="") -> Dict:
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args.output_dir

    eval_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=True)
    os.makedirs(eval_output_dir, exist_ok=True)
    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=collate, drop_last = True
    )

    # multi-gpu evaluate
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        inputs, labels = (batch, batch)
        inputs = inputs.to(args.device)
        labels = labels.to(args.device)

        with torch.no_grad():
            outputs = model(inputs, labels=labels)
            lm_loss = outputs[0]
            eval_loss += lm_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))

    result = {"perplexity": perplexity}

    output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result

In [ ]:
# Main runner

def main(df_trn, df_val):
    args = Args()
    
    if args.should_continue:
        sorted_checkpoints = _sorted_checkpoints(args)
        if len(sorted_checkpoints) == 0:
            raise ValueError("Used --should_continue but no checkpoint was found in --output_dir.")
        else:
            args.model_name_or_path = sorted_checkpoints[-1]

    if (
        os.path.exists(args.output_dir)
        and os.listdir(args.output_dir)
        and args.do_train
        and not args.overwrite_output_dir
        and not args.should_continue
    ):
        raise ValueError(
            "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
                args.output_dir
            )
        )

    # Setup CUDA, GPU & distributed training
    device = torch.device("cuda")
    args.n_gpu = torch.cuda.device_count()
    args.device = device

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        args.local_rank,
        device,
        args.n_gpu,
        bool(args.local_rank != -1),
        args.fp16,
    )

    # Set seed
    set_seed(args)

    config = AutoConfig.from_pretrained(args.config_name, cache_dir=args.cache_dir)
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, cache_dir=args.cache_dir)
    model = AutoModelWithLMHead.from_pretrained(
        args.model_name_or_path,
        from_tf=False,
        config=config,
        cache_dir=args.cache_dir,
    )
    model.to(args.device)
    
    logger.info("Training/evaluation parameters %s", args)

    # Training
    if args.do_train:
        train_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False)

        global_step, tr_loss = train(args, train_dataset, model, tokenizer)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

    # Saving best-practices: if you use save_pretrained for the model and tokenizer, you can reload them using from_pretrained()
    if args.do_train:
        # Create output directory if needed
        os.makedirs(args.output_dir, exist_ok=True)

        logger.info("Saving model checkpoint to %s", args.output_dir)
        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = (
            model.module if hasattr(model, "module") else model
        )  # Take care of distributed/parallel training
        model_to_save.save_pretrained(args.output_dir)
        tokenizer.save_pretrained(args.output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(args, os.path.join(args.output_dir, "training_args.bin"))

        # Load a trained model and vocabulary that you have fine-tuned
        model = AutoModelWithLMHead.from_pretrained(args.output_dir)
        tokenizer = AutoTokenizer.from_pretrained(args.output_dir)
        model.to(args.device)

    # Evaluation
    results = {}
    if args.do_eval and args.local_rank in [-1, 0]:
        checkpoints = [args.output_dir]
        if args.eval_all_checkpoints:
            checkpoints = list(
                os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
            )
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
            prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""

            model = AutoModelWithLMHead.from_pretrained(checkpoint)
            model.to(args.device)
            result = evaluate(args, model, tokenizer, df_trn, df_val, prefix=prefix)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)

    return results

In [ ]:
main(trn_df_all_seasons, test_all_seasons)

03/09/2022 21:01:46 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:882: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
03/09/2022 21:02:11 - INFO - __main__ -   Training/evaluation parameters <__main__.Args object at 0x7f9f4dc92fd0>
03/09/2022 21:02:11 - INFO - __main__ -   Creating features from dataset file at cached
03/09/2022 21:02:34 - INFO - __main__ -   Saving features into cached file cached/gpt2_cached_lm_512
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTo

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1933 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:247: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Iteration:   0%|          | 0/1933 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/1933 [00:00<?, ?it/s]

03/09/2022 21:37:03 - INFO - __main__ -    global_step = 6447, average loss = 1.0166583730083243
03/09/2022 21:37:03 - INFO - __main__ -   Saving model checkpoint to output-small
03/09/2022 21:37:14 - INFO - __main__ -   Evaluate the following checkpoints: ['output-small']
03/09/2022 21:37:16 - INFO - __main__ -   Creating features from dataset file at cached
03/09/2022 21:37:20 - INFO - __main__ -   Saving features into cached file cached/gpt2_cached_lm_512
03/09/2022 21:37:20 - INFO - __main__ -   ***** Running evaluation  *****
03/09/2022 21:37:20 - INFO - __main__ -     Num examples = 1937
03/09/2022 21:37:20 - INFO - __main__ -     Batch size = 4


Evaluating:   0%|          | 0/484 [00:00<?, ?it/s]

03/09/2022 21:38:01 - INFO - __main__ -   ***** Eval results  *****
03/09/2022 21:38:01 - INFO - __main__ -     perplexity = tensor(100.4149)


{'perplexity_': tensor(100.4149)}

Let's generate a conversation

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small')
model = AutoModelWithLMHead.from_pretrained('output-small')

# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')
    # print(new_user_input_ids)

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(
        bot_input_ids, max_length=300,
        pad_token_id=tokenizer.eos_token_id,  
        no_repeat_ngram_size=0,       
        do_sample=True, 
        top_k=50, 
        top_p=0.7,
        temperature = 0.8,
        #add bad words list
        bad_words_ids = [tokenizer(bad_word).input_ids for bad_word in ["!"]]
    )
    
    # pretty print last ouput tokens from bot
    print("GeorgeBot: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:882: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


>> User:Are you ready to head out?
GeorgeBot: Hey, where's Kramer?
>> User:I have no clue, he ate all my food.
GeorgeBot: (over the speaker) Kramer, I'm telling ya, he's a GUY.
>> User:Let's just go get lunch without him.
GeorgeBot: No, Jerry, we don't want you to do that.
>> User:Kramer wont mind.
GeorgeBot: I know, he's a nice guy.
>> User:Aren't you hungry
GeorgeBot: I have a tremendous favor to ask.
